# CONVERSIÓN ARCHIVOS .CSV A .PARQUET

En este *notebook* se transforma el formato original de los archivos *Bicing* de .csv a .parquet, un formato que optimiza el almacenamiento y mejora la rapidez en las consultas. El ejemplo mostrado corresponde al año 2023; para procesar otros años, solamente se deben ajustar las rutas de carga y almacenamiento de los archivos.

In [1]:
import pandas as pd
import os

## Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
%%time
folder_path = "../../1-DATOS/1-DATOS ORIGINALES/BICING/ESTACIONES/2023/"

# Listar todos los archivos en la carpeta que terminen en .csv
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

dataframes = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, low_memory=False)
    dataframes.append(df)

CPU times: total: 1min 20s
Wall time: 1min 23s


In [3]:
%%time 
data_bikes_available = []

for df in dataframes:
    df['last_updated'] = pd.to_datetime(df['last_updated'], unit = 's')
    data_bikes_available.append(df[['station_id', 'num_bikes_available', 'last_updated']])

CPU times: total: 9 s
Wall time: 10.5 s


In [4]:
%%time 
data_bikes_available_wide = []

for df in data_bikes_available:
    df['last_updated'] = pd.to_datetime(df['last_updated'])
    # Eliminar duplicados de 'station_id' y 'last_updated'
    df_cleaned = df.drop_duplicates(subset=['station_id', 'last_updated']).dropna(how='all')
    data_bikes_available_wide.append(df_cleaned.pivot(index='station_id', columns='last_updated', values='num_bikes_available'))

CPU times: total: 51.8 s
Wall time: 55 s


In [ ]:
%%time 
parquet_path_bikes = '../../1-DATOS/2-DATOS PROCESADOS/BICING/CANTIDAD BICICLETAS (MENSUAL)/2023/'

# Guardar los archivos
for i, df in enumerate(data_bikes_available_wide, start=1):
    file_name_bikes = f'2023_{i:02d}_BICICLETAS_ESTACIONES.parquet'
    df.to_parquet(f'{parquet_path_bikes}{file_name_bikes}')

CPU times: total: 1min 4s
Wall time: 1min 10s
